# Практическое задание 3
## Вариант 3
## Задание 1 

Дан набор данных:

| tid              | itemset |
| ---------------- | ------- |
| *t<sub>1</sub>*  | *CE*    |
| *t<sub>2</sub>*  | *CEF*    |
| *t<sub>3</sub>*  | *BE*    |
| *t<sub>4</sub>*  | *ABCDF*    |
| *t<sub>5</sub>*  | *AC*    |
| *t<sub>6</sub>*  | *ACEF*    |
| *t<sub>7</sub>*  | *ABC*    |
| *t<sub>8</sub>*  | *AEF*    |
| *t<sub>9</sub>*  | *ABCDE*   |
| *t<sub>10</sub>* | *DEF*   |

### Алгоритм Apriori

Минимальный уровень поддержки равен **3 / 10** 

1.1. *Самостоятельно* реализуйте алгоритм **Apriori** и продемонстрируйте, как алгоритм перебирает предложенный набор данных. Выведите результат работы алгоритма

1.2. Воспользуйтесь алгоритмом **Apriori** из библиотеки **MLxtend** и выведите результат его работы. Сравните результаты собственной и библиотечной реализации

### Алгоритм FPGrowth

Минимальный уровень поддержки равен **5 / 10**

1.3.  *Самостоятельно* реализуйте алгоритм **FPGrowth** и продемонстрируйте, как алгоритм перебирает предложенный набор данных. Выведите результат работы алгоритма

1.4. Воспользуйтесь алгоритмом **FPGrowth** из библиотеки **MLxtend** и выведите результат его работы. Сравните результаты собственной и библиотечной реализации


In [1]:
import pandas as pd
import numpy as np
import statistics as st
import matplotlib.pyplot as plt
from scipy.stats import norm

In [2]:
from collections import defaultdict

def printResults(itemset,minSupport ):
    if itemset != {}:
        df=pd.DataFrame()
        df['itemset']=[list(item) for item in itemset.keys()]
        df['support']=itemset.values()
        df['length'] = df['itemset'].apply(lambda x: len(x))
        df[''] = [False if val/10<minSupport else True for val in itemset.values()]
        print(df)

def returnItemsWithMinSupport(itemSet, transactionList, minSupport, freqSet): 
        localSet = {}
        for item in itemSet:
                for transaction in transactionList:
                        if item.issubset(transaction):
                                freqSet[item] += 1 
                                localSet.setdefault(item, 0)
                                localSet[item] += 1
        printResults(localSet,minSupport)
        count=len(transactionList)
        localSet = {item:val for item,val in localSet.items() if val/count >= minSupport}
        return localSet


def joinSet(itemSet, length):
        return set([i.union(j) for i in itemSet for j in itemSet if len(i.union(j)) == length])

def getItemSetTransactionList(itemset):
    transactionList = list()
    itemSet = set()
    for record in itemset:
        transaction = frozenset(record)
        transactionList.append(transaction)
        for item in transaction:
            itemSet.add(frozenset([item]))  
    return itemSet, transactionList


def runApriori(dataset, minSupport):
    
    itemSet, transactionList = getItemSetTransactionList(dataset)
    print('Unique elements:',[list(item) for item in itemSet], '\n\nStep 1')

    freqSet = defaultdict(int)
    resultSet = {}
    currentLSet=returnItemsWithMinSupport(itemSet, transactionList, minSupport,freqSet)
    resultSet.update(currentLSet)
    currentLSet = set(currentLSet.keys())
    k = 2
    while currentLSet != set([]):
        
        currentLSet = joinSet(currentLSet, k)
        print('\nStep %d'% k)
        currentCSet = returnItemsWithMinSupport(currentLSet, transactionList, minSupport, freqSet)
        resultSet.update(currentCSet)
        currentLSet = set(currentCSet)
        k += 1
    return resultSet



#if __name__ == "__main__":

itemset={'CE','CEF','BE','ABCDF','AC','ACEF','ABC','AEF','ABCDE','DEF'}
minSupport = 3/10
print('Data:', itemset)
print('Minsup ratio:', minSupport)
    
result = runApriori(itemset, minSupport)
print('\nResult:')
printResults(result, minSupport)


Data: {'CEF', 'BE', 'ABCDF', 'ABC', 'ACEF', 'AC', 'AEF', 'ABCDE', 'DEF', 'CE'}
Minsup ratio: 0.3
Unique elements: [['B'], ['E'], ['C'], ['F'], ['D'], ['A']] 

Step 1
  itemset  support  length      
0     [B]        4       1  True
1     [E]        7       1  True
2     [C]        7       1  True
3     [F]        5       1  True
4     [D]        3       1  True
5     [A]        6       1  True

Step 2
   itemset  support  length       
0   [F, C]        3       2   True
1   [F, B]        1       2  False
2   [C, B]        3       2   True
3   [A, B]        3       2   True
4   [D, B]        2       2  False
5   [E, C]        4       2   True
6   [F, D]        2       2  False
7   [E, D]        2       2  False
8   [A, C]        5       2   True
9   [F, E]        4       2   True
10  [E, B]        2       2  False
11  [C, D]        2       2  False
12  [F, A]        3       2   True
13  [A, D]        2       2  False
14  [A, E]        3       2   True

Step 3
     itemset  support  leng

In [3]:
#Задание 1.2
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

te = TransactionEncoder()
te_ary = te.fit(itemset).transform(itemset)
df = pd.DataFrame(te_ary, columns=te.columns_)
results = apriori(df, min_support=0.3, use_colnames=True)
results['length']= results['itemsets'].apply(lambda x: len(x)) #добавление размера набора
print(results)

    support   itemsets  length
0       0.6        (A)       1
1       0.4        (B)       1
2       0.7        (C)       1
3       0.3        (D)       1
4       0.7        (E)       1
5       0.5        (F)       1
6       0.3     (A, B)       2
7       0.5     (A, C)       2
8       0.3     (A, E)       2
9       0.3     (A, F)       2
10      0.3     (C, B)       2
11      0.4     (E, C)       2
12      0.3     (F, C)       2
13      0.4     (F, E)       2
14      0.3  (A, C, B)       3


In [4]:
min_sup=0.5
F={}
class FPNode(object):
    def __init__(self,names,frequency=1):
        self.name=names
        self.frequency=frequency
        self.ancestors=None
        self.children={}
        self.new=True
    def changeFreq(self):
        self.frequency+=1
    def addNode(self, node):
        if not node.name in node.children:
            self.children[node.name] = node
            node.ancestors = self
    def findNode(self, item):
        try:
            return self.children[item]
        except KeyError:
            return None
            
    def showTree(self, index=1):
        if self.name is not None:
            if self.ancestors and self.ancestors.new==True:
                print('(',end='')
                self.ancestors.new=False
            print('%s:%d:'%(self.name,self.frequency),end='')
            parent = self.ancestors
            p = ''
            while parent is not None and parent.name is not None:
                p = p + parent.name
                parent = parent.ancestors
            print(p[::-1],end='')
            if self.children == {}:
                print('(',end='')
        for child in self.children.values():
            child.showTree(index+1)
            print('),',end='')
class FPTree:
    def __init__(self):
        self.head = FPNode(None)
        self.uniq={}
        self.support_set = defaultdict(int)
    def addNode(self, transaction):
        t_node = self.head
        for item in transaction:
            self.support_set[item] +=1
            n_node = t_node.findNode(item)
            if n_node:
                n_node.changeFreq()
            else:
                n_node = FPNode(item)
                t_node.addNode(n_node)
                self.uniq.setdefault(item,[])
                self.uniq[item].append(n_node)
            t_node = n_node   
    def findNode(s_node):
        for node in self:
            if node.name == s_node:
                node.frequency+=1
    def getAncestors(self):
        for current_node in self:
            pass
        if current_node!=self.head:
            return current_node.ancestors+current_node.name
        return ''
    def showTree(self):
        self.head.showTree()
    def findAncetsetors(self,item):
        if self.name==item:
            return 
    def findSuffixPrefix(self):
        for item,arr_path in self.uniq.items():
            prefix={}
            for path in arr_path:
                node = path
                item=node.name
                frequency=node.frequency
                name_path=''
                node=node.ancestors
                prefix.setdefault(item,[])
                while node is not None and node.name is not None:
                    name_path+=node.name
                    node=node.ancestors
                if name_path!='':
                    prefix[item].append((name_path[::-1],frequency))
            print('sufix: %s, prefix: %s, empty prefix list: %s'%(item,prefix[item],True if prefix[item]== [] else False))
            makeCPTree(prefix[item],item)
            
    def findPrefixWithMinsup(self,suffix):
        for item,node in self.uniq.items():
            support=self.support_set[item]
            if support/10>=min_sup:
                return(item+suffix,support)
             

In [5]:
def getSortingUnique(dataset):
    itemSet = set(item for transaction in dataset for item in transaction)
    combItem = {}
    for item in itemSet:
        for transaction in dataset:
            if transaction.find(item)>-1:
                combItem.setdefault(item, 0)
                combItem[item] += 1
    sorted_dict={}
    sorted_keys=sorted(combItem, key = combItem.get,reverse=True)
    for key in sorted_keys:
        sorted_dict[key]=combItem[key]
    return sorted_dict
def getSortingData(dataset,sorted_dict):
    sort_data=[]
    for transaction in dataset: 
        sort_trans=[]
        for item in sorted_dict.keys():
             if transaction.find(item)>-1:
                    sort_trans.append(item)
        sort_data.append(sort_trans)
    return sort_data
def makeCPTree(prefix_list,suffix):
    tree=FPTree()
    node_sup={}
    for prefix in prefix_list: 
        path=prefix[0]
        frequency=prefix[1]
        for i in range(frequency):
            tree.addNode(path)
    node_sup.setdefault(suffix,[])
    node_sup[suffix].append(tree.findPrefixWithMinsup(suffix))
    print('CP-Tree:')
    tree.showTree()
    print('\n')
    if node_sup[suffix][0] is not None:
        name=node_sup[suffix][0][0]
        F.setdefault(name,0)
        F[name]=node_sup[suffix][0][1]
        print('Add to F: ',(name,F[name]))
    print('\n')
        
def runFPGrowth(dataset,min_support):
    data = getSortingUnique(dataset)
    print('Result sorting unique:',data)
    for item,val in data.items():
        if val/10>=min_support:
            F.setdefault(item,0)
            F[item]=val
            print('Add to F:',(item,val))
    data=getSortingData(dataset,data)
    print('Result sorting data', data)
    tree=FPTree()
    for transaction in data:
        tree.addNode(transaction)
    print('FP-Tree:')
    tree.showTree()
    print(')\n')
    tree.findSuffixPrefix()
    print('Result:')
    printResults(F,min_sup)

In [6]:
runFPGrowth(itemset,min_sup)

Result sorting unique: {'C': 7, 'E': 7, 'A': 6, 'F': 5, 'B': 4, 'D': 3}
Add to F: ('C', 7)
Add to F: ('E', 7)
Add to F: ('A', 6)
Add to F: ('F', 5)
Result sorting data [['C', 'E', 'F'], ['E', 'B'], ['C', 'A', 'F', 'B', 'D'], ['C', 'A', 'B'], ['C', 'E', 'A', 'F'], ['C', 'A'], ['E', 'A', 'F'], ['C', 'E', 'A', 'B', 'D'], ['E', 'F', 'D'], ['C', 'E']]
FP-Tree:
(C:7:(E:4:C(F:1:CE(),A:2:CE(F:1:CEA(),B:1:CEA(D:1:CEAB(),),),),A:3:C(F:1:CA(B:1:CAF(D:1:CAFB(),),),B:1:CA(),),),E:3:(B:1:E(),A:1:E(F:1:EA(),),F:1:E(D:1:EF(),),),)

sufix: C, prefix: [], empty prefix list: True
CP-Tree:




sufix: E, prefix: [('C', 4)], empty prefix list: False
CP-Tree:
(C:4:(),



sufix: F, prefix: [('CE', 1), ('CA', 1), ('CEA', 1), ('EA', 1), ('E', 1)], empty prefix list: False
CP-Tree:
(C:3:(E:2:C(A:1:CE(),),A:1:C(),),E:2:(A:1:E(),),



sufix: B, prefix: [('E', 1), ('CAF', 1), ('CA', 1), ('CEA', 1)], empty prefix list: False
CP-Tree:
(E:1:(),C:3:(A:2:C(F:1:CA(),),E:1:C(A:1:CE(),),),



sufix: A, prefix: [('C', 3), (

In [7]:
#Задание 1.4

from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth

te = TransactionEncoder()
te_ary = te.fit(itemset).transform(itemset)
data = pd.DataFrame(te_ary, columns=te.columns_)
result = fpgrowth(data, min_support=0.5, use_colnames = True)
print(result)


   support itemsets
0      0.7      (E)
1      0.7      (C)
2      0.5      (F)
3      0.6      (A)
4      0.5   (A, C)


## Задание 2
На рисунке представлена классификация различных продуктов. Каждый лист дерева это конкретный продукт, внутренний узел дерева представляет категорию продукта более верхнего уровня.



Был получен следующий набор данных:

| tid  | itemset |
| ---- | ------- |
| *t<sub>1</sub>*  | *ADGHKL*    |
| *t<sub>2</sub>*  | *GIL*    |
| *t<sub>3</sub>*  | *CGHK*    |
| *t<sub>4</sub>*  | *BCDIJL*    |
| *t<sub>5</sub>*  | *CEFJKL*    |
| *t<sub>6</sub>*  | *BDEFIJ*    |
| *t<sub>7</sub>*  | *CDFHKL*    |
| *t<sub>8</sub>*  | *ACEIJKL*    |
| *t<sub>9</sub>*  | *CDGHJ*   |
| *t<sub>10</sub>* | *BCHIKL*   |

Выполните следующие задание:

2.1. Каков размер области поиска наборов элементов, если ограничиваться только наборами, состоящими из простых элементов?

2.2. Минимальный уровень поддержки равен **5 / 10**. Найдите все часто встречающиеся наборы элементов, состоящие только из элементов высокого уровня в таксономии. Имейте в виду, что если в транзакции появляется простой элемент, предполагается, что все его предки высокого уровня также присутствуют в транзакции. При выполнении данного пункта воспользуйтесь собственной реализацией **Apriori**.

In [12]:
#Задание 2.1
simple,_ = getItemSetTransactionList(dataset)
print('Simple set count:',len(simple))


Simple set count: 12


In [9]:
#Задание 2.2
dataset=['ADGHKL', 'GIL','CGHK', 'BCDIJL', 'CEFJKL','BDEFIJ','CDFHKL','ACEIJKL','CDGHJ','BCHIKL']
print('Data:', dataset)
print('Minsup ratio:',0.5)

symbol_map={'A':'O','M':'O','N':'O','I':'Q','P':'Q','G':'G','H':'H','A':'A','B':'M','C':'M','E':'N','D':'N', 'F':'N','I':'I',
            'J':'P','K':'P','L':'P',
            'B':'B','C':'C','D':'D','E':'E','F':'F','J':'J','K':'K','L':'L'}
levels=5
while(levels>1):
    new_data=[]
    for transaction in dataset:
        str=''
        for item, val in symbol_map.items():
            if transaction.find(item)>-1:
                str=str+val
        new_data.append(str)
    levels-=1
    dataset=new_data  
result = runApriori(new_data,0.5)


Data: ['ADGHKL', 'GIL', 'CGHK', 'BCDIJL', 'CEFJKL', 'BDEFIJ', 'CDFHKL', 'ACEIJKL', 'CDGHJ', 'BCHIKL']
Minsup ratio: 0.5
Unique elements: [['B'], ['E'], ['J'], ['C'], ['K'], ['F'], ['D'], ['L'], ['H'], ['I'], ['G'], ['A']] 

Step 1
   itemset  support  length       
0      [B]        3       1  False
1      [E]        3       1  False
2      [J]        5       1   True
3      [C]        7       1   True
4      [K]        6       1   True
5      [F]        3       1  False
6      [D]        5       1   True
7      [L]        7       1   True
8      [H]        5       1   True
9      [I]        5       1   True
10     [G]        4       1  False
11     [A]        2       1  False

Step 2
   itemset  support  length       
0   [J, H]        1       2  False
1   [L, K]        5       2   True
2   [I, J]        3       2  False
3   [I, C]        3       2  False
4   [L, C]        5       2   True
5   [L, D]        3       2  False
6   [C, D]        3       2  False
7   [C, K]        5       

In [10]:
print('Symbol map:',symbol_map)
print('R data:',new_data)
print('\nResult:')
printResults(result,0.5)

Symbol map: {'A': 'A', 'M': 'O', 'N': 'O', 'I': 'I', 'P': 'Q', 'G': 'G', 'H': 'H', 'B': 'B', 'C': 'C', 'E': 'E', 'D': 'D', 'F': 'F', 'J': 'J', 'K': 'K', 'L': 'L'}
R data: ['AGHDKL', 'IGL', 'GHCK', 'IBCDJL', 'CEFJKL', 'IBEDFJ', 'HCDFKL', 'AICEJKL', 'GHCDJ', 'IHBCKL']

Result:
  itemset  support  length      
0     [J]        5       1  True
1     [C]        7       1  True
2     [K]        6       1  True
3     [D]        5       1  True
4     [L]        7       1  True
5     [H]        5       1  True
6     [I]        5       1  True
7  [L, K]        5       2  True
8  [L, C]        5       2  True
9  [C, K]        5       2  True
